In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from openpyxl import load_workbook

In [5]:
"Maestro "+datetime.now().strftime("%Y%m%d")

'Maestro 20230125'

In [2]:
file_i = 'Maestro Enero 2023.xlsx'
master_wb = load_workbook(file_i)
sheet = master_wb.worksheets[0]

In [3]:
n_row = sheet.max_row
n_col = sheet.max_column
n_row, n_col

(1936, 14)

In [4]:
color = []  

for i in range(1,n_row):
    cell1 = sheet.cell(column=1, row=i)
    cell2 = sheet.cell(column=2, row=i)
    id = cell1.value
    bgColor = cell2.fill.bgColor.index
    
    if (bgColor != '00000000'): #or (fgColor != '00000000'):
        color.append((id, bgColor))

print(color)


[(393, 'FFFFFF00'), (429, 'FF481D32'), (491, 'FF481D32'), (524, 'FF481D32'), (525, 'FF481D32'), (526, 'FF481D32'), (527, 'FF481D32'), (528, 'FF481D32'), (529, 'FF481D32'), (530, 'FF481D32'), (531, 'FF481D32'), (532, 'FF481D32'), (533, 'FF481D32'), (534, 'FF481D32'), (535, 'FF481D32'), (536, 'FF481D32'), (537, 'FF481D32'), (538, 'FF481D32'), (539, 'FF481D32'), (540, 'FF481D32'), (541, 'FF481D32'), (542, 'FF481D32')]


In [5]:
def get_sheet_changes(file_name):
    master_wb = load_workbook(file_name)            # we use the function load_workbook from openpyxl
    sheet = master_wb.worksheets[0]                 # select the first sheet
    n_row, n_col = sheet.max_row, sheet.max_column  # define the rows and cols of the sheet
    headers = [c.value for c in sheet[1] if c.value is not None]
    changes = []

    for i in range(1,n_row):
        asset_changes = []
        cell = sheet.cell(column=2, row=i)
        asset_color = cell.fill.bgColor.index
        if (asset_color != '00000000'):
            effective_sheet = sheet[i][2:]              # we are interested in other columns
            for j, col in enumerate(effective_sheet):     
                col_color = col.fill.bgColor.index
                if (col_color != '00000000'):           # if cell has color then we save the changes
                    asset_changes.append({'value':col.value,'column':headers[col.column-1]})
            changes.append({'id':sheet.cell(column=1, row=i).value ,'changes':asset_changes})

    return changes
            


In [6]:
changes = get_sheet_changes(file_i)
changes

[{'id': 393,
  'changes': [{'value': 'ENEX', 'column': 'alias1'},
   {'value': 'Especialista 4', 'column': 'alias2'}]},
 {'id': 429,
  'changes': [{'value': 1, 'column': 'active'},
   {'value': 'GEMIN', 'column': 'alias1'},
   {'value': 'Especialista 6', 'column': 'alias2'}]},
 {'id': 491,
  'changes': [{'value': 0, 'column': 'active'},
   {'value': 'JYD M', 'column': 'alias1'},
   {'value': 'Especialista 2', 'column': 'alias2'}]},
 {'id': 524,
  'changes': [{'value': 0, 'column': 'active'},
   {'value': 'P&J', 'column': 'alias1'},
   {'value': 'Especialista 1', 'column': 'alias2'}]},
 {'id': 525,
  'changes': [{'value': 0, 'column': 'active'},
   {'value': 'P&J', 'column': 'alias1'},
   {'value': 'Especialista 6', 'column': 'alias2'}]},
 {'id': 526,
  'changes': [{'value': 0, 'column': 'active'},
   {'value': 'P&J', 'column': 'alias1'},
   {'value': 'Especialista 1', 'column': 'alias2'}]},
 {'id': 527,
  'changes': [{'value': 0, 'column': 'active'},
   {'value': 'P&J', 'column': 'alia

In [165]:
def set_sheet_changes(changes, file_name):

    df_master = pd.read_excel(file_name, sheet_name=0)

    # Here we will do the changes in the line (row)
    for change in changes:#[18:]:

        t_e = datetime.utcnow().replace(hour=3, minute=0, second=0) - timedelta(seconds=1)
        t_s = datetime.utcnow().replace(hour=3, minute=0, second=0)

        idx = df_master.loc[df_master["id"] == change["id"]].index[0] # saving the index where there are changes


        # we get the amount of void rows
        void_rows_counter = 0
        while(df_master.isna().loc[idx + 1 + void_rows_counter, "id"]):
            void_rows_counter += 1



        for col_change in change["changes"]:    # iterating in all changes of certain row

            print(change["id"], col_change["column"], col_change["value"], void_rows_counter)

            # If the previous value was different than the current one then we do the change (we compare in a string form)
            if (str(df_master.loc[df_master["id"] == change["id"], col_change["column"]].values[0]) != str(col_change["value"])):
                df_master.loc[df_master["id"] == change["id"], col_change["column"]] = col_change["value"]



                ct          =   "c_" if str(col_change["column"]) == "active" else "t_"
                t_col_v     =   ct + col_change["column"] + "_v"
                t_col_ds    =   ct + col_change["column"] + "_ds"
                t_col_de    =   ct + col_change["column"] + "_de"


                # if the asset doesnt have void rows bellow then we check the line
                if void_rows_counter == 0 :
                    print("no tengo filas iniciales")
                    if df_master.loc[df_master["id"] == change["id"], t_col_v].isna().bool():
                        df_master.loc[df_master["id"] == change["id"], t_col_v] = col_change["value"]

                    elif (df_master.loc[df_master["id"] == change["id"], t_col_v].values[0] != col_change["value"]):
                        print(str(df_master.loc[df_master["id"] == change["id"], t_col_v].values[0]), str(col_change["value"]))
                        void_row = pd.DataFrame([[np.nan for i in range(len(df_master.columns))]], columns=df_master.columns)
                        void_row.loc[0, t_col_v] = col_change["value"] # here we fill the cell

                        df_master = pd.concat([df_master.iloc[:idx+1+void_rows_counter], void_row, df_master.iloc[idx+1+void_rows_counter:]]).reset_index(drop=True)
                        void_rows_counter += 1

                        df_master.loc[idx + void_rows_counter - 1, t_col_de] = t_e.strftime("%Y%m%d %H:%M:%S")
                        df_master.loc[idx + void_rows_counter, t_col_ds] = t_s.strftime("%Y%m%d %H:%M:%S")

                        print("se agrega fila")


                else :
                    print("si tengo filas iniciales")

                    # we check the last added row and if the cell is void we search where we have to put the value
                    if df_master.isna().loc[idx + void_rows_counter, t_col_v]:
                        for i in range(void_rows_counter):

                            if  (df_master.loc[idx + i, t_col_v] == col_change["value"]): # (df_master.isna().loc[idx + 1 + i, t_col_v]) and
                                print("Estoy vacio y el anterior si es lo mismo")
                                break

                            else: #(df_master.loc[idx + i, t_col_v] != col_change["value"]):
                                df_master.loc[idx + 1 + i, t_col_v] = col_change["value"]

                                # here we put the end and start time of changes
                                df_master.loc[idx + i, t_col_de] = t_e.strftime("%Y%m%d %H:%M:%S")
                                df_master.loc[idx + i + 1, t_col_ds] = t_s.strftime("%Y%m%d %H:%M:%S")

                                print("Estoy vacio y el anterior no es lo mismo")
                                print(str(df_master.loc[idx + i, t_col_v]), str(col_change["value"]))
                                break


                    # if the last value is the same that the change then we dont do anything
                    elif str(df_master.loc[idx + void_rows_counter, t_col_v].values[0]) == str(col_change["value"]) :
                        pass

                    # if the last cell is filled then we add a void row and we put the value
                    else:
                        void_row = pd.DataFrame([[np.nan for i in range(len(df_master.columns))]], columns=df_master.columns)
                        void_row.loc[0, t_col_v] = col_change["value"] # here we fill the cell

                        df_master = pd.concat([df_master.iloc[:idx+void_rows_counter], void_row, df_master.iloc[idx+void_rows_counter:]]).reset_index(drop=True)
                        void_rows_counter += 1
                        # here we put the end and start time of changes
                        df_master.loc[idx + void_rows_counter - 1, t_col_de] = t_e.strftime("%Y%m%d %H:%M:%S")
                        df_master.loc[idx + void_rows_counter, t_col_ds] = t_s.strftime("%Y%m%d %H:%M:%S")

                        print("se agrega fila")


    df_master.to_excel("test.xlsx", index = False)
    df_master.to_csv("test.csv")

In [166]:
file_f = 'maestro_20230110.xlsx'
set_sheet_changes(changes, file_f)

393 alias1 ENEX 0
no tengo filas iniciales
EDS F ENEX
se agrega fila
393 alias2 Especialista 4 1
429 active 1 1
si tengo filas iniciales
Estoy vacio y el anterior si es lo mismo
429 alias1 GEMIN 1
429 alias2 Especialista 6 1
491 active 0 0
no tengo filas iniciales
1.0 0
se agrega fila
491 alias1 JYD M 1
491 alias2 Especialista 2 1
524 active 0 0
no tengo filas iniciales
524 alias1 P&J 0
524 alias2 Especialista 1 0
525 active 0 0
no tengo filas iniciales
525 alias1 P&J 0
525 alias2 Especialista 6 0
526 active 0 0
no tengo filas iniciales
1.0 0
se agrega fila
526 alias1 P&J 1
526 alias2 Especialista 1 1
527 active 0 0
no tengo filas iniciales
1.0 0
se agrega fila
527 alias1 P&J 1
527 alias2 Especialista 1 1
528 active 0 0
no tengo filas iniciales
1.0 0
se agrega fila
528 alias1 P&J 1
528 alias2 Especialista 1 1
529 active 0 0
no tengo filas iniciales
1.0 0
se agrega fila
529 alias1 P&J 1
529 alias2 Especialista 1 1
530 active 0 0
no tengo filas iniciales
530 alias1 P&J 0
530 alias2 Espec

In [128]:
df_master.loc[df_master["id"] == 524, "t_alias1_v"].values[0]

'P&J'

In [24]:

df_master = pd.read_excel(file_f, sheet_name=0)
#idx = df_master.loc[df_master["id"] == 393].index[0]
if df_master.loc[df_master["id"] == 393, "t_alias1_v"].isna().bool() == False:
    print("Hola")

Hola


In [24]:
df = pd.DataFrame({'Name': ['John', 'Jane', None, 'Bob'], 'Age': [25, 30, 35, 40]})
df

,Name,Age
0,John,25
1,Jane,30
2,None,35
3,Bob,40


In [10]:
df = pd.DataFrame({'Name': ['John', 'Jane', 'Bob', 'John'], 'Age': [25, 30, 35, 40]})

# Crear una nueva fila con todos los valores como NaN
new_row = pd.DataFrame([[np.nan for i in range(len(df.columns))]], columns=df.columns)
new_row.loc[0, "Age"] = 15

# Concatenar la nueva fila con el DataFrame original
df = pd.concat([df.iloc[:2], new_row, df.iloc[2:]]).reset_index(drop=True)
df

,Name,Age
0,John,25.0
1,Jane,30.0
2,NaN,15.0
3,Bob,35.0
4,John,40.0


In [17]:
aux = df[df["Name"]=="Bob"]
#aux.loc[3][1]
aux.index

Int64Index([3], dtype='int64')

In [23]:
i = df.loc[df['Name'] == 'John'].index[0]
df.isna().loc[i, "Name"]

False

In [41]:
arr = ['John', 'Jane', 'Bob', 'Jonas']
condition1 = np.char.find(arr, 'Jo') > -1
condition2 = np.char.find(arr, 'ob') > -1
result = np.extract(np.logical_or(condition1, condition2), arr)
result
#result = arr[indices[0]]
#result

array(['John', 'Bob', 'Jonas'], dtype='<U5')

In [152]:
fecha_y_hora_actual = datetime.utcnow().replace(hour=0+input,minute=0,seconds=0)
fecha_y_hora_actual
fecha_y_hora_termino = fecha_y_hora_actual -timedelta(seconds=1)

datetime.datetime(2023, 1, 25, 13, 6, 45, 546181)

In [153]:
fecha_y_hora_formateada = fecha_y_hora_actual.strftime("%Y%m%d %H:%M:%S")
fecha_y_hora_formateada

'20230125 13:06:45'

In [156]:
datetime.utcnow().replace(hour=10, minute=0, second=0)

datetime.datetime(2023, 1, 25, 10, 0, 0, 158469)

In [7]:
df

,Name,Age
0,John,25
1,Jane,30
2,None,35
3,Bob,40


In [8]:
df.loc[:2]

,Name,Age
0,John,25
1,Jane,30
2,None,35


In [13]:
"John" in df["Name"].unique()

True

In [18]:
df.append({"Name": "Jason"}, ignore_index = 1)

/tmp/ipykernel_99610/4125989797.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append({"Name": "Jason"}, ignore_index = 1)


,Name,Age
0,John,25.0
1,Jane,30.0
2,None,35.0
3,Bob,40.0
4,Jason,NaN


In [40]:
df1 = pd.concat([df,pd.DataFrame({"Name": ["Jason"], "Weight": [81]})]).reset_index(drop=True)

In [41]:
df1

,Name,Age,Weight
0,John,25.0,65
1,Jane,30.0,55
2,None,35.0,72
3,Bob,40.0,67
4,Jason,NaN,81


In [36]:
df["Weight"] = [65, 55, 72, 67]

In [37]:
df

,Name,Age,Weight
0,John,25,65
1,Jane,30,55
2,None,35,72
3,Bob,40,67
